In [2]:
import asyncio
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker
from sqlalchemy.future import select
from statsmodels.api import OLS
import pandas as pd
from sqlalchemy import and_

from fleecekmbackend.db.models import Author, Answer, Rating
from fleecekmbackend.core.config import DATABASE_URL

# Setting up the async engine and session
engine = create_async_engine(DATABASE_URL)
async_session = sessionmaker(engine, expire_on_commit=False, class_=AsyncSession)


async def fetch_data():
    async with async_session() as session:
        # Select answers authored by 'meta-llama/Meta-Llama-3-70B-Instruct' with their ratings
        results = await session.execute(
            select(Answer, Rating)
            .join(Rating, Answer.id == Rating.answer_id)
            .join(Author, Author.id == Answer.author_id)
            .where(Author.model == "meta-llama/Meta-Llama-3-70B-Instruct")
        )
        return results.all()
    
data = await fetch_data()
data

2024-05-06 21:40:42,335 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-05-06 21:40:42,336 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-06 21:40:42,338 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-05-06 21:40:42,339 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-06 21:40:42,340 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-05-06 21:40:42,341 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-06 21:40:42,343 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-06 21:40:42,353 INFO sqlalchemy.engine.Engine SELECT answer.id, answer.question_id, answer.author_id, answer.setting, answer.timestamp, answer.text, rating.id AS id_1, rating.text AS text_1, rating.value, rating.answer_id, rating.author_id AS author_id_1, rating.timestamp AS timestamp_1 
FROM answer INNER JOIN rating ON answer.id = rating.answer_id INNER JOIN author ON author.id = answer.author_id 
WHERE author.model = %s
2024-05-06 21:40:42,354 INFO sqlalchemy.engine.Engine [generated